In [1]:
import timm
import torchsummary
import glob
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
from torch.autograd import Variable
from collections import OrderedDict
from timm.data import Dataset, create_loader, resolve_data_config,  FastCollateMixup, mixup_batch, AugMixDataset
from timm.models import create_model, resume_checkpoint, convert_splitbn_model, apply_test_time_pool
from timm.utils import *
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler
from munch import Munch
import yaml
import sys
from gd import Generator, Discriminator
from datetime import datetime
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
# from apex import amp
# from apex.parallel import DistributedDataParallel as DDP
# from apex.parallel import convert_syncbn_model
has_apex = False

In [3]:
# GPU Device
gpu_id = '0,1,2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device " , use_cuda)

GPU device  True


In [4]:
torch.backends.cudnn.benchmark = True

In [5]:
with open('config/train.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
args = Munch(config)
args.prefetcher = not args.no_prefetcher
args.distributed = False
args.device = 'cuda'
args.world_size = 3
args.rank = 0
logging.info('Training with a single process on %d GPUs.' % args.num_gpu)

INFO:root:Training with a single process on 3 GPUs.


In [6]:
if args.distributed:
    args.num_gpu = 1
    args.device = 'cuda:%d' % args.local_rank
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend='nccl', init_method='env://', rank=args.rank, world_size=args.world_size)
    args.world_size = torch.distributed.get_world_size()
    args.rank = torch.distributed.get_rank()
    assert args.rank >= 0

In [7]:
torch.manual_seed(args.seed + args.rank)

In [8]:
model_ns = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
model_ns = model_ns.cuda()
model_ns = torch.nn.DataParallel(model_ns)

In [9]:
model_raw = timm.create_model('tf_efficientnet_b1', pretrained=True)
model_raw = model_raw.cuda()
model_raw = torch.nn.DataParallel(model_raw)

In [10]:
model_g = Generator(args, img_size=240, max_conv_dim=256)
model_g = model_g.cuda()

In [11]:
optimizer = create_optimizer(args, model_g)

In [12]:
use_amp = False
if has_apex and args.amp:
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    model_raw =  amp.initialize(model_raw)
    model_ns = amp.initialize(model_ns)
    use_amp = True
if args.local_rank == 0:
    logging.info('NVIDIA APEX {}. AMP {}.'.format(
        'installed' if has_apex else 'not installed', 'on' if use_amp else 'off'))

INFO:root:NVIDIA APEX not installed. AMP off.


In [13]:
# optionally resume from a checkpoint
resume_state = {}
resume_epoch = None
if args.resume:
    resume_state, resume_epoch = resume_checkpoint(model_g, args.resume)
if resume_state and not args.no_resume_opt:
    if 'optimizer' in resume_state:
        if args.local_rank == 0:
            logging.info('Restoring Optimizer state from checkpoint')
        optimizer.load_state_dict(resume_state['optimizer'])
    if use_amp and 'amp' in resume_state and 'load_state_dict' in amp.__dict__:
        if args.local_rank == 0:
            logging.info('Restoring NVIDIA AMP state from checkpoint')
        amp.load_state_dict(resume_state['amp'])
del resume_state

INFO:root:Loaded checkpoint './output/train/20200625-003932-tf_efficientnet_b1-240/checkpoint-0.pth.tar' (epoch 0)
INFO:root:Restoring Optimizer state from checkpoint


In [14]:
model_g = torch.nn.DataParallel(model_g)

In [15]:
train_dir = '/home/data/imagenet/train'
val_dir = '/home/data/imagenet/val'
data_config = resolve_data_config(vars(args), model=model_g, verbose=args.local_rank == 0)

INFO:root:Data processing configuration for current model + dataset:
INFO:root:	input_size: (3, 240, 240)
INFO:root:	interpolation: bicubic
INFO:root:	mean: (0.485, 0.456, 0.406)
INFO:root:	std: (0.229, 0.224, 0.225)
INFO:root:	crop_pct: 0.882


In [16]:
num_aug_splits = 0

In [17]:
if args.distributed:
    if args.sync_bn:
        assert not args.split_bn
        try:
            if has_apex:
                model = convert_syncbn_model(model)
            else:
                model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
            if args.local_rank == 0:
                logging.info(
                    'Converted model to use Synchronized BatchNorm. WARNING: You may have issues if using '
                    'zero initialized BN layers (enabled by default for ResNets) while sync-bn enabled.')
        except Exception as e:
            logging.error('Failed to enable Synchronized BatchNorm. Install Apex or Torch >= 1.1')
    if has_apex:
        model = DDP(model, delay_allreduce=True)
    else:
        if args.local_rank == 0:
            logging.info("Using torch DistributedDataParallel. Install NVIDIA Apex for Apex DDP.")
        model = DDP(model, device_ids=[args.local_rank])  # can use device str in Torch >= 1.1
    # NOTE: EMA model does not need to be wrapped by DDP


In [18]:
lr_scheduler, num_epochs = create_scheduler(args, optimizer)

In [19]:
resume_state = {}
resume_epoch = None
start_epoch = 0
if args.start_epoch is not None:
    # a specified start_epoch will always override the resume epoch
    start_epoch = args.start_epoch
elif resume_epoch is not None:
    start_epoch = resume_epoch
if lr_scheduler is not None and start_epoch > 0:
    lr_scheduler.step(start_epoch)

In [20]:
if args.local_rank == 0:
    logging.info('Scheduled epochs: {}'.format(num_epochs))

INFO:root:Scheduled epochs: 200


In [21]:
train_dataset = Dataset(train_dir)
val_dataset = Dataset(val_dir, load_bytes=False, class_map='')

In [22]:
param_count = sum([m.numel() for m in model_g.parameters()])
logging.info('Model created, param count: %d' % (param_count))

INFO:root:Model created, param count: 4192780


In [23]:
collate_fn = None
if args.prefetcher and args.mixup > 0:
    assert not num_aug_splits  # collate conflict (need to support deinterleaving in collate mixup)
    collate_fn = FastCollateMixup(args.mixup, args.smoothing, args.num_classes)

In [24]:
model_raw, test_time_pool = apply_test_time_pool(model_raw, data_config, args)

In [25]:
train_loader = create_loader(
        train_dataset,
        input_size=data_config['input_size'],
        batch_size=args.batch_size,
        is_training=True,
        use_prefetcher=args.prefetcher,
        re_prob=args.reprob,
        re_mode=args.remode,
        re_count=args.recount,
        re_split=args.resplit,
        color_jitter=args.color_jitter,
        auto_augment=args.aa,
        num_aug_splits=num_aug_splits,
        interpolation=args.train_interpolation,
        mean=data_config['mean'],
        std=data_config['std'],
        num_workers=args.workers,
        distributed=args.distributed,
        collate_fn=collate_fn,
        pin_memory=args.pin_mem,
        use_multi_epochs_loader=args.use_multi_epochs_loader
    )

In [26]:
crop_pct = 1.0 if test_time_pool else data_config['crop_pct']
val_loader = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=args.batch_size,
    is_training=False,
    use_prefetcher=args.prefetcher,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=args.workers,
    crop_pct=crop_pct,
    pin_memory=args.pin_mem,
    tf_preprocessing=args.tf_preprocessing)

In [27]:
if args.jsd:
    assert num_aug_splits > 1  # JSD only valid with aug splits set
    train_loss_fn = JsdCrossEntropy(num_splits=num_aug_splits, smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.mixup > 0.:
    # smoothing is handled with mixup label transform
    train_loss_fn = SoftTargetCrossEntropy()
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.smoothing:
    train_loss_fn = LabelSmoothingCrossEntropy(smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
else:
    train_loss_fn = nn.CrossEntropyLoss()
    validate_loss_fn = train_loss_fn

gan_loss_fn = nn.BCELoss()

In [28]:
eval_metric = args.eval_metric
best_metric = None
best_epoch = None
saver = None
output_dir = ''
if args.local_rank == 0:
    output_base = args.output if args.output else './output'
    exp_name = '-'.join([
        datetime.now().strftime("%Y%m%d-%H%M%S"),
        args.model,
        str(data_config['input_size'][-1])
    ])
    output_dir = get_outdir(output_base, 'train', exp_name)
    decreasing = True if eval_metric == 'loss' else False
    saver = CheckpointSaver(checkpoint_dir=output_dir, decreasing=decreasing)

In [29]:
model_ema = None
if args.model_ema:
    # Important to create EMA model after cuda(), DP wrapper, and AMP but before SyncBN and DDP wrapper
    model_ema = ModelEma(
        model,
        decay=args.model_ema_decay,
        device='cpu' if args.model_ema_force_cpu else '',
        resume=args.resume)

In [30]:
def train_epoch(epoch, model_g, model_raw, model_ns, loader, optimizer, loss_fn, gan_loss_fn, args,
               lr_scheduler=None, saver=None, output_dir='', use_amp=False, model_ema=None):

    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()
    losses_g = AverageMeter()
    losses_ce = AverageMeter()
    losses_kd = AverageMeter()
    
    model_g.train()
    model_ns.eval()
    model_raw.eval()

    real, fake = 1, 0
    
    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
    for batch_idx, (inputs, target) in enumerate(loader):
        last_batch = batch_idx == last_idx
        data_time_m.update(time.time() - end)
        z = torch.randn(args.batch_size, 1, 15, 15, device='cuda')
        g_out = model_g(z)
        
        # G train
        model_g.zero_grad()
#         label.fill_(real)
#         g_d_out = model_d(inputs+g_out)
#         loss_g = gan_loss_fn(g_d_out, label)
        
        # KD train
        inputs_z = inputs + g_out
        output, traj_raw = model_raw(inputs_z)
        with torch.no_grad():
            out_ns, traj_ns = model_ns(inputs)
            out_ns = out_ns.detach()
            
        p_s = F.log_softmax(output/args.T, dim=1)
        p_t = F.softmax(out_ns/args.T, dim=1)
        loss_kd = F.kl_div(p_s, p_t, size_average=False) * (args.T ** 2) / output.shape[0]
        
        # CE train
        loss_ce = loss_fn(output, target)
        
        # overall loss
        loss = args.lambda_kd * loss_kd + args.lambda_ce * loss_ce
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if not args.distributed:
#             losses_d.update(loss_d.item(), inputs.size(0))
#             losses_g.update(loss_g.item(), inputs.size(0))
            losses_kd.update(loss_kd.item(), inputs.size(0))
            losses_ce.update(loss_ce.item(), inputs.size(0))
            losses_m.update(loss.item(), inputs.size(0))

        if model_ema is not None:
            model_ema.update(model)
        num_updates += 1

        batch_time_m.update(time.time() - end)
        if last_batch or batch_idx % args.log_interval == 0:
            lrl = [param_group['lr'] for param_group in optimizer.param_groups]
            lr = sum(lrl) / len(lrl)

            if args.distributed:
                reduced_loss = reduce_tensor(loss.data, args.world_size)
                losses_m.update(reduced_loss.item(), inputs.size(0))

            if args.local_rank == 0:
                logging.info(
                    'Train: {} [{:>4d}/{} ({:>3.0f}%)]  '
                    'Loss: {loss.val:>9.6f} ({loss.avg:>6.4f})  '
#                     'Loss_d: {loss_d.val:>9.6f} ({loss_d.avg:>6.4f})  '
#                     'Loss_g: {loss_g.val:>9.6f} ({loss_g.avg:>6.4f})  '
                    'Loss_kd: {loss_kd.val:>9.6f} ({loss_kd.avg:>6.4f})  '
                    'Loss_ce: {loss_ce.val:>9.6f} ({loss_ce.avg:>6.4f})  '
                    'Time: {batch_time.val:.3f}s, {rate:>7.2f}/s  '
                    '({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'LR: {lr:.3e}  '
                    'Data: {data_time.val:.3f} ({data_time.avg:.3f})'.format(
                        epoch,
                        batch_idx, len(loader),
                        100. * batch_idx / last_idx,
                        loss=losses_m,
#                         loss_d=losses_d,
#                         loss_g=losses_g,
                        loss_kd=losses_kd,
                        loss_ce=losses_ce,
                        batch_time=batch_time_m,
                        rate=inputs.size(0) * args.world_size / batch_time_m.val,
                        rate_avg=inputs.size(0) * args.world_size / batch_time_m.avg,
                        lr=lr,
                        data_time=data_time_m))

                if args.save_images and output_dir:
                    torchvision.utils.save_image(
                        inputs_z,
                        os.path.join(output_dir, 'train-batch-%d.jpg' % batch_idx),
                        padding=0,
                        normalize=True)

        if saver is not None and args.recovery_interval and (
                last_batch or (batch_idx + 1) % args.recovery_interval == 0):
            saver.save_recovery(
                model, optimizer, args, epoch, model_ema=model_ema, use_amp=use_amp, batch_idx=batch_idx)

        if lr_scheduler is not None:
            lr_scheduler.step_update(num_updates=num_updates, metric=losses_m.avg)

        end = time.time()
        # end for

    if hasattr(optimizer, 'sync_lookahead'):
        optimizer.sync_lookahead()

    return OrderedDict([('loss', losses_m.avg)])

In [31]:
def val_epoch(model_raw, model, val_loader, criterion, args):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model_raw.eval()
    model.eval()
    with torch.no_grad():
        # warmup, reduce variability of first batch time, especially for comparing torchscript vs non
        end = time.time()
        for i, (inputs, target) in enumerate(val_loader):
            if args.no_prefetcher:
                target = target.cuda()
                inputs = inputs.cuda()
            z = torch.randn(inputs.shape[0], 1, 15, 15, device='cuda')
            
            out = model(z)
            # synthesizing input + generator
            inputs_out = inputs + out
            # compute output
            output, foward_list = model_raw(inputs_out)
            loss = criterion(output, target)
            
            # measure accuracy and record loss
            acc1, acc5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc1.item(), inputs.size(0))
            top5.update(acc5.item(), inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.log_freq == 0:
                logging.info(
                    'Test: [{0:>4d}/{1}]  '
                    'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '
                    'Acc@1: {top1.val:>7.3f} ({top1.avg:>7.3f})  '
                    'Acc@5: {top5.val:>7.3f} ({top5.avg:>7.3f})'.format(
                        i, len(val_loader), batch_time=batch_time,
                        rate_avg=inputs.size(0) / batch_time.avg,
                        loss=losses, top1=top1, top5=top5))

    results = OrderedDict(
        top1=round(top1.avg, 4), top1_err=round(100 - top1.avg, 4),
        top5=round(top5.avg, 4), top5_err=round(100 - top5.avg, 4),
        param_count=round(param_count / 1e6, 2),
        img_size=data_config['input_size'][-1],
        cropt_pct=crop_pct,
        interpolation=data_config['interpolation'])

    logging.info(' * Acc@1 {:.3f} ({:.3f}) Acc@5 {:.3f} ({:.3f})'.format(
       results['top1'], results['top1_err'], results['top5'], results['top5_err']))
    return results

In [ ]:
for epoch in range(start_epoch, num_epochs):
    if args.distributed:
        train_loader.sampler.set_epoch(epoch)

    train_metrics = train_epoch(
        epoch, model_g, model_raw, model_ns, train_loader, optimizer, train_loss_fn, gan_loss_fn, args,
        lr_scheduler=lr_scheduler, saver=saver, output_dir=output_dir,
        use_amp=use_amp, model_ema=model_ema)

    eval_metrics = val_epoch(model_raw, model_g, val_loader, validate_loss_fn, args)

    if model_ema is not None and not args.model_ema_force_cpu:
        if args.distributed and args.dist_bn in ('broadcast', 'reduce'):
            distribute_bn(model_ema, args.world_size, args.dist_bn == 'reduce')


    if lr_scheduler is not None:
        # step LR for next epoch
        lr_scheduler.step(epoch + 1, eval_metrics[eval_metric])
        
    update_summary(
        epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'),
        write_header=best_metric is None)

    if saver is not None:
    # save proper checkpoint with eval metric
        save_metric = eval_metrics[eval_metric]
        best_metric, best_epoch = saver.save_checkpoint(
            model_g, optimizer, args,
            epoch=epoch, model_ema=model_ema, metric=save_metric, use_amp=use_amp)

/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
INFO:root:Train: 1 [   0/6100 (  0%)]  Loss:  5.276267 (5.2763)  Loss_kd:  0.348140 (0.3481)  Loss_ce:  1.794868 (1.7949)  Time: 20.053s,   31.42/s  (20.053s,   31.42/s)  LR: 1.000e-04  Data: 4.200 (4.200)
INFO:root:Train: 1 [  50/6100 (  1%)]  Loss:  5.252110 (5.2731)  Loss_kd:  0.347956 (0.3543)  Loss_ce:  1.772553 (1.7303)  Time: 1.112s,  566.35/s  (1.578s,  399.28/s)  LR: 1.000e-04  Data: 0.001 (0.084)
INFO:root:Train: 1 [ 100/6100 (  2%)]  Loss:  5.275858 (5.2804)  Loss_kd:  0.353982 (0.3544)  Loss_ce:  1.736035 (1.7361)  Time: 1.110s,  567.72/s  (1.347s,  467.61/s)  LR: 1.000e-04  Data: 0.001 (0.043)
INFO:root:Train: 1 [ 150/6100 (  2%)]  Loss:  5.347467 (5.2629)  Loss_kd:  0.356035 (0.3531)  Loss_ce:  1.787115 (1.7315)  Time: 1.111s,  566.88/s  (1.270s,  496.12/s)  LR: 1.

INFO:root:Train: 1 [1950/6100 ( 32%)]  Loss:  5.210876 (5.2574)  Loss_kd:  0.344078 (0.3527)  Loss_ce:  1.770100 (1.7301)  Time: 1.113s,  566.14/s  (1.125s,  560.00/s)  LR: 1.000e-04  Data: 0.001 (0.003)
INFO:root:Train: 1 [2000/6100 ( 33%)]  Loss:  5.489964 (5.2572)  Loss_kd:  0.369536 (0.3527)  Loss_ce:  1.794599 (1.7302)  Time: 1.112s,  566.40/s  (1.125s,  560.14/s)  LR: 1.000e-04  Data: 0.001 (0.003)
INFO:root:Train: 1 [2050/6100 ( 34%)]  Loss:  5.243854 (5.2565)  Loss_kd:  0.354922 (0.3526)  Loss_ce:  1.694637 (1.7301)  Time: 1.113s,  565.99/s  (1.124s,  560.28/s)  LR: 1.000e-04  Data: 0.001 (0.003)
INFO:root:Train: 1 [2100/6100 ( 34%)]  Loss:  5.288067 (5.2564)  Loss_kd:  0.349567 (0.3527)  Loss_ce:  1.792400 (1.7299)  Time: 1.114s,  565.78/s  (1.124s,  560.42/s)  LR: 1.000e-04  Data: 0.001 (0.003)
INFO:root:Train: 1 [2150/6100 ( 35%)]  Loss:  5.417606 (5.2562)  Loss_kd:  0.365958 (0.3526)  Loss_ce:  1.758026 (1.7298)  Time: 1.109s,  568.02/s  (1.124s,  560.56/s)  LR: 1.000e-04  

INFO:root:Train: 1 [3900/6100 ( 64%)]  Loss:  5.341830 (5.2552)  Loss_kd:  0.360665 (0.3526)  Loss_ce:  1.735178 (1.7294)  Time: 1.112s,  566.64/s  (1.119s,  562.93/s)  LR: 1.000e-04  Data: 0.001 (0.002)
INFO:root:Train: 1 [3950/6100 ( 65%)]  Loss:  5.245465 (5.2552)  Loss_kd:  0.349749 (0.3526)  Loss_ce:  1.747975 (1.7294)  Time: 1.113s,  566.28/s  (1.119s,  562.97/s)  LR: 1.000e-04  Data: 0.001 (0.002)
INFO:root:Train: 1 [4000/6100 ( 66%)]  Loss:  5.018976 (5.2552)  Loss_kd:  0.345294 (0.3526)  Loss_ce:  1.566032 (1.7295)  Time: 1.112s,  566.33/s  (1.119s,  563.00/s)  LR: 1.000e-04  Data: 0.001 (0.002)
INFO:root:Train: 1 [4050/6100 ( 66%)]  Loss:  5.527436 (5.2557)  Loss_kd:  0.373173 (0.3526)  Loss_ce:  1.795704 (1.7298)  Time: 1.112s,  566.50/s  (1.119s,  563.04/s)  LR: 1.000e-04  Data: 0.001 (0.002)
INFO:root:Train: 1 [4100/6100 ( 67%)]  Loss:  5.132126 (5.2553)  Loss_kd:  0.341636 (0.3526)  Loss_ce:  1.715763 (1.7297)  Time: 1.112s,  566.34/s  (1.119s,  563.07/s)  LR: 1.000e-04  

INFO:root:Train: 1 [5950/6100 ( 98%)]  Loss:  5.344646 (5.2548)  Loss_kd:  0.366044 (0.3525)  Loss_ce:  1.684207 (1.7296)  Time: 1.111s,  566.88/s  (1.117s,  563.90/s)  LR: 1.000e-04  Data: 0.001 (0.002)
INFO:root:Train: 1 [6000/6100 ( 98%)]  Loss:  5.366744 (5.2548)  Loss_kd:  0.364773 (0.3525)  Loss_ce:  1.719016 (1.7296)  Time: 1.112s,  566.33/s  (1.117s,  563.92/s)  LR: 1.000e-04  Data: 0.001 (0.002)
INFO:root:Train: 1 [6050/6100 ( 99%)]  Loss:  5.186093 (5.2548)  Loss_kd:  0.344458 (0.3525)  Loss_ce:  1.741510 (1.7298)  Time: 1.113s,  566.15/s  (1.117s,  563.93/s)  LR: 1.000e-04  Data: 0.001 (0.002)
INFO:root:Train: 1 [6099/6100 (100%)]  Loss:  5.276394 (5.2548)  Loss_kd:  0.344298 (0.3525)  Loss_ce:  1.833411 (1.7298)  Time: 1.364s,  462.01/s  (1.117s,  563.93/s)  LR: 1.000e-04  Data: 0.256 (0.002)
INFO:root:Test: [   0/239]  Time: 5.281s (5.281s,   39.76/s)  Loss:  0.5095 (0.5095)  Acc@1:  88.095 ( 88.095)  Acc@5:  97.619 ( 97.619)
INFO:root:Test: [  10/239]  Time: 0.326s (0.789

INFO:root:Train: 2 [1000/6100 ( 16%)]  Loss:  5.187089 (5.2464)  Loss_kd:  0.354365 (0.3519)  Loss_ce:  1.643439 (1.7278)  Time: 1.112s,  566.67/s  (1.121s,  562.16/s)  LR: 1.000e-04  Data: 0.001 (0.006)
INFO:root:Train: 2 [1050/6100 ( 17%)]  Loss:  4.944619 (5.2441)  Loss_kd:  0.331376 (0.3517)  Loss_ce:  1.630859 (1.7270)  Time: 1.114s,  565.78/s  (1.120s,  562.34/s)  LR: 1.000e-04  Data: 0.001 (0.006)
INFO:root:Train: 2 [1100/6100 ( 18%)]  Loss:  5.311405 (5.2441)  Loss_kd:  0.357476 (0.3517)  Loss_ce:  1.736647 (1.7272)  Time: 1.115s,  565.09/s  (1.120s,  562.49/s)  LR: 1.000e-04  Data: 0.001 (0.005)
INFO:root:Train: 2 [1150/6100 ( 19%)]  Loss:  5.279851 (5.2440)  Loss_kd:  0.357943 (0.3517)  Loss_ce:  1.700418 (1.7271)  Time: 1.113s,  566.11/s  (1.120s,  562.63/s)  LR: 1.000e-04  Data: 0.001 (0.005)
INFO:root:Train: 2 [1200/6100 ( 20%)]  Loss:  5.453130 (5.2438)  Loss_kd:  0.366625 (0.3517)  Loss_ce:  1.786884 (1.7270)  Time: 1.113s,  566.00/s  (1.119s,  562.76/s)  LR: 1.000e-04  

...

INFO:root:Train: 4 [ 850/6100 ( 14%)]  Loss:  5.072076 (5.2473)  Loss_kd:  0.340963 (0.3519)  Loss_ce:  1.662448 (1.7280)  Time: 1.113s,  566.26/s  (1.119s,  563.11/s)  LR: 1.000e-05  Data: 0.001 (0.007)
INFO:root:Train: 4 [ 900/6100 ( 15%)]  Loss:  5.174829 (5.2469)  Loss_kd:  0.344904 (0.3519)  Loss_ce:  1.725788 (1.7276)  Time: 1.112s,  566.74/s  (1.119s,  563.24/s)  LR: 1.000e-05  Data: 0.001 (0.006)
INFO:root:Train: 4 [ 950/6100 ( 16%)]  Loss:  5.225054 (5.2461)  Loss_kd:  0.336219 (0.3519)  Loss_ce:  1.862860 (1.7271)  Time: 1.111s,  566.98/s  (1.118s,  563.38/s)  LR: 1.000e-05  Data: 0.001 (0.006)
INFO:root:Train: 4 [1000/6100 ( 16%)]  Loss:  5.257072 (5.2443)  Loss_kd:  0.353004 (0.3518)  Loss_ce:  1.727031 (1.7264)  Time: 1.112s,  566.35/s  (1.118s,  563.49/s)  LR: 1.000e-05  Data: 0.001 (0.006)
INFO:root:Train: 4 [1050/6100 ( 17%)]  Loss:  5.356981 (5.2464)  Loss_kd:  0.358711 (0.3519)  Loss_ce:  1.769875 (1.7273)  Time: 1.117s,  563.84/s  (1.118s,  563.63/s)  LR: 1.000e-05  

INFO:root:Train: 4 [2900/6100 ( 48%)]  Loss:  5.134595 (5.2465)  Loss_kd:  0.342725 (0.3519)  Loss_ce:  1.707349 (1.7277)  Time: 1.113s,  566.02/s  (1.115s,  565.12/s)  LR: 1.000e-05  Data: 0.001 (0.003)
INFO:root:Train: 4 [2950/6100 ( 48%)]  Loss:  5.019041 (5.2467)  Loss_kd:  0.339299 (0.3519)  Loss_ce:  1.626051 (1.7278)  Time: 1.114s,  565.72/s  (1.115s,  565.14/s)  LR: 1.000e-05  Data: 0.001 (0.003)
INFO:root:Train: 4 [3000/6100 ( 49%)]  Loss:  5.359919 (5.2469)  Loss_kd:  0.357968 (0.3519)  Loss_ce:  1.780239 (1.7280)  Time: 1.113s,  566.15/s  (1.115s,  565.16/s)  LR: 1.000e-05  Data: 0.001 (0.003)
INFO:root:Train: 4 [3050/6100 ( 50%)]  Loss:  5.241621 (5.2469)  Loss_kd:  0.356843 (0.3519)  Loss_ce:  1.673189 (1.7281)  Time: 1.113s,  566.25/s  (1.115s,  565.17/s)  LR: 1.000e-05  Data: 0.001 (0.003)
INFO:root:Train: 4 [3100/6100 ( 51%)]  Loss:  5.131267 (5.2473)  Loss_kd:  0.340612 (0.3519)  Loss_ce:  1.725148 (1.7283)  Time: 1.113s,  566.00/s  (1.115s,  565.18/s)  LR: 1.000e-05  

INFO:root:Train: 4 [4950/6100 ( 81%)]  Loss:  5.277558 (5.2489)  Loss_kd:  0.349736 (0.3520)  Loss_ce:  1.780201 (1.7287)  Time: 1.114s,  565.52/s  (1.114s,  565.41/s)  LR: 1.000e-05  Data: 0.001 (0.002)
INFO:root:Train: 4 [5000/6100 ( 82%)]  Loss:  5.042574 (5.2487)  Loss_kd:  0.346263 (0.3520)  Loss_ce:  1.579946 (1.7286)  Time: 1.112s,  566.45/s  (1.114s,  565.41/s)  LR: 1.000e-05  Data: 0.001 (0.002)
INFO:root:Train: 4 [5050/6100 ( 83%)]  Loss:  5.231630 (5.2483)  Loss_kd:  0.363357 (0.3520)  Loss_ce:  1.598065 (1.7284)  Time: 1.112s,  566.78/s  (1.114s,  565.41/s)  LR: 1.000e-05  Data: 0.001 (0.002)
INFO:root:Train: 4 [5100/6100 ( 84%)]  Loss:  5.419349 (5.2480)  Loss_kd:  0.362193 (0.3520)  Loss_ce:  1.797424 (1.7283)  Time: 1.113s,  566.21/s  (1.114s,  565.42/s)  LR: 1.000e-05  Data: 0.001 (0.002)
INFO:root:Train: 4 [5150/6100 ( 84%)]  Loss:  5.575677 (5.2484)  Loss_kd:  0.378404 (0.3520)  Loss_ce:  1.791632 (1.7285)  Time: 1.114s,  565.65/s  (1.114s,  565.42/s)  LR: 1.000e-05  

INFO:root:Test: [ 230/239]  Time: 0.324s (0.356s,  589.12/s)  Loss:  1.8873 (0.8967)  Acc@1:  56.190 ( 78.969)  Acc@5:  85.714 ( 94.257)
INFO:root: * Acc@1 78.986 (21.014) Acc@5 94.306 (5.694)
INFO:root:Current checkpoints:
 ('./output/train/20200625-133839-tf_efficientnet_b1-240/checkpoint-4.pth.tar', 78.986)
 ('./output/train/20200625-133839-tf_efficientnet_b1-240/checkpoint-3.pth.tar', 78.962)
 ('./output/train/20200625-133839-tf_efficientnet_b1-240/checkpoint-1.pth.tar', 78.926)
 ('./output/train/20200625-133839-tf_efficientnet_b1-240/checkpoint-2.pth.tar', 78.916)

INFO:root:Train: 5 [   0/6100 (  0%)]  Loss:  4.968460 (4.9685)  Loss_kd:  0.330562 (0.3306)  Loss_ce:  1.662840 (1.6628)  Time: 6.240s,  100.96/s  (6.240s,  100.96/s)  LR: 1.000e-05  Data: 4.359 (4.359)
INFO:root:Train: 5 [  50/6100 (  1%)]  Loss:  5.155859 (5.2383)  Loss_kd:  0.350301 (0.3505)  Loss_ce:  1.652851 (1.7333)  Time: 1.115s,  564.79/s  (1.228s,  513.03/s)  LR: 1.000e-05  Data: 0.002 (0.087)
INFO:root:Train

INFO:root:Train: 5 [1900/6100 ( 31%)]  Loss:  5.346860 (5.2495)  Loss_kd:  0.361856 (0.3519)  Loss_ce:  1.728296 (1.7305)  Time: 1.114s,  565.74/s  (1.117s,  564.05/s)  LR: 1.000e-05  Data: 0.001 (0.003)
INFO:root:Train: 5 [1950/6100 ( 32%)]  Loss:  5.256540 (5.2489)  Loss_kd:  0.352992 (0.3519)  Loss_ce:  1.726617 (1.7302)  Time: 1.114s,  565.42/s  (1.117s,  564.09/s)  LR: 1.000e-05  Data: 0.001 (0.003)
INFO:root:Train: 5 [2000/6100 ( 33%)]  Loss:  5.332098 (5.2488)  Loss_kd:  0.361255 (0.3519)  Loss_ce:  1.719545 (1.7299)  Time: 1.113s,  566.07/s  (1.117s,  564.14/s)  LR: 1.000e-05  Data: 0.001 (0.003)
INFO:root:Train: 5 [2050/6100 ( 34%)]  Loss:  5.034050 (5.2484)  Loss_kd:  0.340233 (0.3519)  Loss_ce:  1.631719 (1.7296)  Time: 1.114s,  565.54/s  (1.117s,  564.18/s)  LR: 1.000e-05  Data: 0.001 (0.003)
INFO:root:Train: 5 [2100/6100 ( 34%)]  Loss:  5.248077 (5.2490)  Loss_kd:  0.345843 (0.3519)  Loss_ce:  1.789647 (1.7298)  Time: 1.115s,  564.93/s  (1.117s,  564.22/s)  LR: 1.000e-05  